In [1]:
import pandas as pd
import json


from data_loading import DataLoader_Data
from data_cleaning import DatasetCleaner 

# Load the dataset
file_path = '/Users/alaaeddinalia/Desktop/bachelor_arbeit /Rumor_verification/data/raw/English_train.json' 
loader = DataLoader_Data(file_path)
data = loader.data



cleaner = DatasetCleaner()

# Run the cleaning process
cleaned_data = cleaner.remove_irrelevant_rumors(data)


total_rumors_before_cleaning = len(data)  # Total number of rumors before cleaning
total_rumors_after_cleaning = 0  # Total number of rumors after cleaning (still have valid tweets)
total_rumors_affected = 0  # Number of rumors where tweets were removed

# Track cleaning results
for rumor in data:
    timeline = rumor.get("timeline", [])
    total_tweets_before = len(timeline)
    cleaned_timeline = [entry for entry in timeline if cleaner.normalize_text(entry[2]) not in cleaner.issue_texts]
    total_tweets_after = len(cleaned_timeline)
    
    # Count rumors that still have valid tweets after cleaning
    if total_tweets_after > 0:
        total_rumors_after_cleaning += 1
    
    # Count rumors affected
    if total_tweets_before > total_tweets_after:
        total_rumors_affected += 1


summary_row = {
    "cleaning_approach": "Filtered out irrelevant rumors based on issue texts",
    "total_rumors_affected": total_rumors_affected,
    "total_rumors_before_cleaning": total_rumors_before_cleaning,
    "total_rumors_after_cleaning": total_rumors_after_cleaning
}

df_summary = pd.DataFrame([summary_row])


print("Table: Data Cleaning Summary for Rumors")
display(df_summary)



Table: Data Cleaning Summary for Rumors


,cleaning_approach,total_rumors_affected,total_rumors_before_cleaning,total_rumors_after_cleaning
0,Filtered out irrelevant rumors based on issue ...,8,96,88


In [2]:
import pandas as pd
import json


from data_loading import DataLoader_Data
from data_cleaning import DatasetCleaner 

# Load the dataset using DataLoader_Data class
file_path = '/Users/alaaeddinalia/Desktop/bachelor_arbeit /Rumor_verification/data/raw/English_train.json' 
loader = DataLoader_Data(file_path)
data = loader.data


cleaner = DatasetCleaner()

# Run the cleaning 
cleaned_data = cleaner.remove_irrelevant_rumors(data)

total_tweets_before_cleaning = 0
total_tweets_after_cleaning = 0
total_timelines_affected = 0  

# Track cleaning results 
for rumor in data:
    timeline = rumor.get("timeline", [])
    total_tweets_before = len(timeline)
    cleaned_timeline = [entry for entry in timeline if cleaner.normalize_text(entry[2]) not in cleaner.issue_texts]
    total_tweets_after = len(cleaned_timeline)
    

    total_tweets_before_cleaning += total_tweets_before
    total_tweets_after_cleaning += total_tweets_after
    
    # Count the number of tweets affected in the timeline 
    total_timelines_affected += (total_tweets_before - total_tweets_after)


summary_row = {
    "cleaning_approach": "Filtered out irrelevant rumors based on issue texts",
    "total_timelines_affected": total_timelines_affected,  
    "total_timeline_before_cleaning": total_tweets_before_cleaning,
    "total_timeline_after_cleaning": total_tweets_after_cleaning
}

df_summary = pd.DataFrame([summary_row])


print("Table: Data Cleaning Summary")
display(df_summary)

Table: Data Cleaning Summary


,cleaning_approach,total_timelines_affected,total_timeline_before_cleaning,total_timeline_after_cleaning
0,Filtered out irrelevant rumors based on issue ...,3912,21805,17893


In [3]:
import pandas as pd
import json

from data_loading import DataLoader_Data
from data_cleaning import DatasetCleaner
from preprocessor import Preprocessor 

# Load the dataset
file_path = '/Users/alaaeddinalia/Desktop/bachelor_arbeit /Rumor_verification/data/raw/English_train.json' 
loader = DataLoader_Data(file_path)
data = loader.data


cleaner = DatasetCleaner()


cleaned_data = cleaner.remove_irrelevant_rumors(data)


total_tweets_before_preprocessing = 0
preprocessing_results = []


preprocessing_steps = [
    {"name": "URLs Removed", "preprocessor": Preprocessor(language='english', remove_urls=True)},
    {"name": "Noise Words Removed", "preprocessor": Preprocessor(language='english', remove_noise_words=True)},
    {"name": "Special Characters Removed", "preprocessor": Preprocessor(language='english', remove_special_characters=True)},
    {"name": "Stopwords Removed", "preprocessor": Preprocessor(language='english', remove_stopwords=True)},
]

# Process each timeline in the dataset
for step in preprocessing_steps:
    preprocessor = step["preprocessor"]
    total_tweets_affected = 0
    total_tweets_before = 0

    # Track preprocessing for each timeline
    for rumor in cleaned_data:
        timeline = rumor.get("timeline", [])
        total_tweets_before += len(timeline)
        
      
        preprocessed_timeline = [preprocessor.preprocess_text(tweet[2]) for tweet in timeline]
        
        # Count how many tweets were affected 
        tweets_affected = sum(1 for original, preprocessed in zip(timeline, preprocessed_timeline) if original[2] != preprocessed)
        total_tweets_affected += tweets_affected

    
    preprocessing_results.append({
        "approach_used": step["name"],
        "mode_of_effect": "Removed" if "Removed" in step["name"] else "Modified",
        "tweets_affected_in_timeline": total_tweets_affected,
        "total_tweets_in_timeline": total_tweets_before
    })


df_preprocessing_summary = pd.DataFrame(preprocessing_results)


print("Table: Data Preprocessing Outcome")
display(df_preprocessing_summary)



Table: Data Preprocessing Outcome


,approach_used,mode_of_effect,tweets_affected_in_timeline,total_tweets_in_timeline
0,URLs Removed,Removed,13794,17893
1,Noise Words Removed,Removed,17859,17893
2,Special Characters Removed,Removed,17866,17893
3,Stopwords Removed,Removed,17889,17893
